In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import datetime

In [3]:
train_origin = pd.read_csv("./elo-merchant-category-recommendation/train.csv")
test_origin = pd.read_csv("./elo-merchant-category-recommendation/test.csv")
historical_transactions = pd.read_csv("./elo-merchant-category-recommendation/historical_transactions.csv",parse_dates=['purchase_date'])
merchants = pd.read_csv("./elo-merchant-category-recommendation/merchants.csv")
new_merchant_transactions = pd.read_csv("./elo-merchant-category-recommendation/new_merchant_transactions.csv",parse_dates=['purchase_date'])


In [4]:
train = train_origin
test = test_origin

# Basic Data Exploration

In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201917 entries, 0 to 201916
Data columns (total 6 columns):
first_active_month    201917 non-null datetime64[ns]
card_id               201917 non-null object
feature_1             201917 non-null category
feature_2             201917 non-null category
feature_3             201917 non-null category
target                201917 non-null float64
dtypes: category(3), datetime64[ns](1), float64(1), object(1)
memory usage: 5.2+ MB


In [5]:
train_origin.head(1)

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283


In [6]:
historical_transactions.head(1)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37


In [7]:
merchants.head(1)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.4,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0


In [8]:
new_merchant_transactions.head(1)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19


# Data Processing

## Transform Data Types

In [9]:
train["feature_1"] = train["feature_1"].astype("category")
train["feature_2"] = train["feature_2"].astype("category")
train["feature_3"] = train["feature_3"].astype("category")

In [10]:
historical_transactions["authorized_flag"].unique()

array(['Y', 'N'], dtype=object)

In [11]:
# historical_transactions["authorized_flag"] = historical_transactions["authorized_flag"].astype("category")
# historical_transactions["category_1"] = historical_transactions["category_1"].astype("category")
# historical_transactions["category_3"] = historical_transactions["category_3"].astype("category")
# historical_transactions["merchant_id"] = historical_transactions["merchant_id"].astype("category")
# historical_transactions["category_2"] = historical_transactions["category_2"].astype("category")
# historical_transactions["city_id"] = historical_transactions["city_id"].astype("category")

In [12]:
# Transform Y & N to 1 and 0
def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df


In [13]:
historical_transactions = binarize(historical_transactions)
new_merchant_transactions = binarize(new_merchant_transactions)

In [14]:
train['first_active_month'] = pd.to_datetime(train['first_active_month'])
test['first_active_month'] = pd.to_datetime(test['first_active_month'])


In [15]:
historical_transactions.head(1)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37


## Fill in Missing Values

### Create a column to indicate the missing values

In [16]:
historical_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29112361 entries, 0 to 29112360
Data columns (total 14 columns):
authorized_flag         int64
card_id                 object
city_id                 int64
category_1              int64
installments            int64
category_3              object
merchant_category_id    int64
merchant_id             object
month_lag               int64
purchase_amount         float64
purchase_date           datetime64[ns]
category_2              float64
state_id                int64
subsector_id            int64
dtypes: datetime64[ns](1), float64(2), int64(8), object(3)
memory usage: 3.0+ GB


In [17]:
historical_transactions.loc[:,"categoty_3_missing"] = np.array(historical_transactions.loc[:,"category_3"].isnull()).astype(int)
historical_transactions.loc[:,"categoty_2_missing"] = np.array(historical_transactions.loc[:,"category_2"].isnull()).astype(int)
historical_transactions.loc[:,"merchant_id_missing"] = np.array(historical_transactions.loc[:,"merchant_id"].isnull()).astype(int)

new_merchant_transactions.loc[:,"categoty_3_missing"] = np.array(new_merchant_transactions.loc[:,"category_3"].isnull()).astype(int)
new_merchant_transactions.loc[:,"categoty_2_missing"] = np.array(new_merchant_transactions.loc[:,"category_2"].isnull()).astype(int)
new_merchant_transactions.loc[:,"merchant_id_missing"] = np.array(new_merchant_transactions.loc[:,"merchant_id"].isnull()).astype(int)



In [19]:
historical_transactions.loc[:,"category_2"].fillna('0.0', inplace =True)
historical_transactions.loc[:,"category_3"].fillna('M', inplace =True)
historical_transactions.loc[:,"merchant_id"].fillna('0', inplace =True)

new_merchant_transactions.loc[:,"category_2"].fillna('0.0', inplace =True)
new_merchant_transactions.loc[:,"category_3"].fillna('M', inplace =True)
new_merchant_transactions.loc[:,"merchant_id"].fillna('0', inplace =True)


# Feature Engineering


In [20]:
historical_transactions['month_diff'] = ((datetime.datetime.today() - historical_transactions['purchase_date']).dt.days)//30
historical_transactions['month_diff'] += historical_transactions['month_lag']

new_merchant_transactions['month_diff'] = ((datetime.datetime.today() - new_merchant_transactions['purchase_date']).dt.days)//30
new_merchant_transactions['month_diff'] += new_merchant_transactions['month_lag']


In [22]:
historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
new_merchant_transactions['purchase_month'] = new_merchant_transactions['purchase_date'].dt.month
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_merchant_transactions = pd.get_dummies(new_merchant_transactions, columns=['category_2', 'category_3'])

In [ ]:
authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
non_authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

## Aggregation

In [25]:
def aggregate_transactions(data):
    
    data.loc[:, 'purchase_date'] = pd.DatetimeIndex(data['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    agg_func = {
    'category_1': ['sum', 'mean'],
    'category_2_0.0': ['mean'],
    'category_2_1.0': ['mean'],
    'category_2_2.0': ['mean'],
    'category_2_3.0': ['mean'],
    'category_2_4.0': ['mean'],
    'category_2_5.0': ['mean'],
    'category_3_M': ['mean'],
    'category_3_A': ['mean'],
    'category_3_B': ['mean'],
    'category_3_C': ['mean'],
    'merchant_id': ['nunique'],
    'merchant_category_id': ['nunique'],
    "merchant_id_missing":["mean"],
    "categoty_2_missing":["mean"],
    "categoty_3_missing":["mean"],
    'state_id': ['nunique'],
    'city_id': ['nunique'],
    'subsector_id': ['nunique'],
    'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
    'installments': ['sum', 'mean', 'max', 'min', 'std'],
    'purchase_month': ['mean', 'max', 'min', 'std'],
    'purchase_date': [np.ptp, 'min', 'max'],
    'month_lag': ['mean', 'max', 'min', 'std'],
    'month_diff': ['mean']
    }
    
    agg_data = data.groupby(['card_id']).agg(agg_func)
    agg_data.columns = ['_'.join(col).strip() for col in agg_data.columns.values]
    agg_data.reset_index(inplace=True)
    
    df = (data.groupby('card_id').size() \
          .reset_index(name='transactions_count'))
    
    agg_data = pd.merge(df, agg_data, on='card_id', how='left')
    
    return agg_data

In [26]:
new_merchant_transactions.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount',
       'purchase_date', 'state_id', 'subsector_id', 'categoty_3_missing',
       'categoty_2_missing', 'merchant_id_missing', 'month_diff',
       'purchase_month', 'category_2_1.0', 'category_2_2.0', 'category_2_3.0',
       'category_2_4.0', 'category_2_5.0', 'category_2_0.0', 'category_3_A',
       'category_3_B', 'category_3_C', 'category_3_M'],
      dtype='object')

In [27]:
authorize = aggregate_transactions(authorized_transactions)
non_authorize = aggregate_transactions(non_authorized_transactions)
new_merchant = aggregate_transactions(new_merchant_transactions)

/Users/lush/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/lush/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [28]:
authorize.columns = ['auth_' + c if c != 'card_id' else c for c in authorize.columns]
non_authorize.columns = ['non_auth_' + c if c != 'card_id' else c for c in non_authorize.columns]
new_merchant.columns = ['new_' + c if c != 'card_id' else c for c in new_merchant.columns]


In [29]:
authorize.to_csv("authorize.csv")
non_authorize.to_csv("non_authorize.csv")
new_merchant.to_csv("new_merchant.csv")

# Model Building

In [51]:
train["feature_1"] = train["feature_1"].astype("int")
train["feature_2"] = train["feature_2"].astype("int")
train["feature_3"] = train["feature_3"].astype("int")

In [52]:
train = pd.get_dummies(train, columns=["feature_1","feature_2", "feature_3"])


In [53]:
data_mergerd = train.merge(authorize,how = "left", on = "card_id")
data_mergerd = data_mergerd.merge(non_authorize,how = "left", on = "card_id")
data_mergerd = data_mergerd.merge(new_merchant,how = "left", on = "card_id")

In [54]:
data_mergerd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 201916
Columns: 142 entries, first_active_month to new_month_diff_mean
dtypes: datetime64[ns](1), float64(116), int64(14), object(1), uint8(10)
memory usage: 206.8+ MB


In [57]:
data_mergerd = data_mergerd.fillna(0)

In [58]:
X = data_mergerd[[column for column in data_mergerd.columns if column not in ["card_id","target","first_active_month"]]]
Y = pd.DataFrame(data_mergerd["target"])


## Linear Regression

### Split Train & Test set

In [59]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size = 0.33)

### Build Linear Regression Model

In [64]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(train_x,train_y)
linear_model.score(test_x,test_y)
pre_lm = linear_model.predict(test_x)

## Random Forest

In [63]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(train_x,train_y)
RF_model.score(test_x,test_y)
pre_RF = RF_model.predict(test_x)

### Parameters Tuning with Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 3000, num = 20)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 210, num = 21)]
max_depth.append(None)
min_samples_split = [2, 5, 10, 20]
min_samples_leaf = [1, 2, 4, 6]
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
RF = RandomForestRegressor()
RF_RandomSearch = RandomizedSearchCV(estimator = RF, param_distributions = random_grid, n_iter = 200, cv = ts_split.split(origin_train_x), verbose=2, random_state=42, n_jobs = -1)
RF_RandomSearch.fit(origin_train_x_dummy, origin_train_y)
RF_RandomSearch_predict = RF_RandomSearch.predict(origin_test_x_dummy)
RF_RandomSearch_score = RF_RandomSearch.score(origin_test_x_dummy,origin_test_y)

